## Loading the queries & qrels

In [2]:
from constants import load_pickle_file, clinical_trials_qrels_file, clinical_trials_queries_file_gg
import numpy as np

"""
    Get the queries & qrels
    either calculate it online or get it from the pickle file
"""

# dict { query_id : list of {doc_id,rel} }
# qrels = get_qrels(use_cache=True)
qrels = load_pickle_file(clinical_trials_qrels_file)

# list of { query_id , result : [ list of doc_id ] }
queries = load_pickle_file(clinical_trials_queries_file_gg)

qrels_docs_dict = {}
for qid, l in qrels.items():
    qrels_docs_dict.update({qid: [item['doc_id'] for item in l]})

loading : ../../pickle_files/clinical_trials/title+summary/clinical_trials_qrels.pickle
loading : ../../pickle_files/clinical_trials/clinical_queries.pickle


## Intersected documents

In [3]:
def calculate_relevant_count(retrieved_docs: list, query_docs: list) -> int:
    """
    Find the number of intersected documents between `retrieved_docs` and `query_docs`

    Args:
        retrieved_docs: Al list of doc_id that returned from matching
        query_docs: A list of doc_id belonging to a qid from qrel file

    Returns:
        Number of shared results
    """
    intersect_values = np.intersect1d(retrieved_docs, query_docs)
    matched_count = len(intersect_values)
    return matched_count


## Presision@K

In [4]:
def precision_at_k(relevant_docs, retrieved_docs, k=10) -> float | int:
    """
    Calculates Precision@k

    Args:
        relevant_docs: A dictionary mapping query IDs to a list of relevant document IDs.
        retrieved_docs: A dictionary mapping query IDs to a list of retrieved document IDs, ranked by relevance.
        k: The number of top retrieved documents to consider (default 10).

    Returns:
        The Precision value.
    """

    retrieved = retrieved_docs[:k]
    num_retrieved = len(retrieved)
    num_retrieved_relevant = calculate_relevant_count(retrieved, relevant_docs)
    return num_retrieved_relevant / num_retrieved if num_retrieved > 0 else 0


## Recall@K

In [5]:
def c_recall(relevant_docs, retrieved_docs, k=10):
    """
    Calculates Recall@k

    Args:
        relevant_docs: A dictionary mapping query IDs to a list of relevant document IDs.
        retrieved_docs: A dictionary mapping query IDs to a list of retrieved document IDs, ranked by relevance.
        k: The number of top retrieved documents to consider (default 10).

    Returns:
        A dictionary mapping query IDs to recall@k scores.
    """
    recall_scores = {}
    for query_id, relevant in relevant_docs.items():
        if query_id in retrieved_docs:
            retrieved = retrieved_docs[query_id][:k]  # Consider only top k retrieved documents
            num_relevant = len(relevant)
            num_retrieved_relevant = calculate_relevant_count(retrieved, relevant)
            recall_scores[query_id] = num_retrieved_relevant / num_relevant if num_relevant > 0 else 0
    return recall_scores


c_recall(qrels_docs_dict, queries, k=10)

{1: 0.17647058823529413,
 2: 0.0364963503649635,
 3: 0.08333333333333333,
 4: 0.05263157894736842,
 5: 0.05555555555555555,
 6: 0.1111111111111111,
 7: 0.01951219512195122,
 8: 0.0,
 9: 0.08064516129032258,
 11: 0.21052631578947367,
 12: 0.05128205128205128,
 13: 0.0,
 14: 0.2857142857142857,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.02631578947368421,
 19: 0.0,
 20: 0.0,
 21: 0.014925373134328358,
 22: 0.00980392156862745,
 23: 0.03333333333333333,
 24: 0.05555555555555555,
 25: 0.0,
 26: 0.0,
 27: 0.03571428571428571,
 28: 0.0,
 29: 0.0,
 30: 0.0}

## Average precision

In [6]:
def get_rel_from_list(list_of_rel, doc_id) -> int:
    """Get the rel for a given doc_id from a list of {'doc_id': 'NCT00445783', 'rel': 1}, ..."""

    for rel in list_of_rel:
        if rel['doc_id'] == doc_id:
            return rel['rel']

    return 0




def average_precision(retrieved: list, relevant: list):
    p_sum = 0
    num_of_relevant = 0
    for i in range(10):
        k = i + 1

        # get the doc_id's for the current
        relevant_docs = [doc['doc_id'] for doc in relevant]

        p_at_k = precision_at_k(relevant_docs, retrieved, k)
        print(f'P@{k} : {p_at_k}')
        # get the k document id
        k_doc_id = retrieved[:k][-1]

        # get the rel(k)
        rel_at_k = get_rel_from_list(relevant, k_doc_id)

        if rel_at_k > 0:
            num_of_relevant += rel_at_k
        p_sum += p_at_k * rel_at_k

    return p_sum / num_of_relevant if num_of_relevant > 0 else 0


# MAP

In [7]:
def mean_average_precision(queries: dict, qrels: dict) -> float | int:
    ap_sum = 0
    for qid, query_results in queries.items():
        if qid == 10:
            continue
        print(f'query number {qid} : ')
        val = average_precision(queries[qid], qrels[qid])
        print(f'******* Average Precision : {val}')
        ap_sum += val
        print('-------------------------------------')
    return ap_sum / len(queries)

mean_average_precision(queries, qrels)

query number 1 : 
P@1 : 1.0
P@2 : 0.5
P@3 : 0.6666666666666666
P@4 : 0.5
P@5 : 0.4
P@6 : 0.5
P@7 : 0.42857142857142855
P@8 : 0.375
P@9 : 0.3333333333333333
P@10 : 0.3
******* Average Precision : 0.7222222222222222
-------------------------------------
query number 2 : 
P@1 : 1.0
P@2 : 1.0
P@3 : 1.0
P@4 : 1.0
P@5 : 1.0
P@6 : 0.8333333333333334
P@7 : 0.7142857142857143
P@8 : 0.625
P@9 : 0.5555555555555556
P@10 : 0.5
******* Average Precision : 1.0
-------------------------------------
query number 3 : 
P@1 : 1.0
P@2 : 0.5
P@3 : 0.3333333333333333
P@4 : 0.5
P@5 : 0.4
P@6 : 0.3333333333333333
P@7 : 0.2857142857142857
P@8 : 0.25
P@9 : 0.2222222222222222
P@10 : 0.2
******* Average Precision : 0.8333333333333334
-------------------------------------
query number 4 : 
P@1 : 0.0
P@2 : 0.0
P@3 : 0.0
P@4 : 0.25
P@5 : 0.2
P@6 : 0.16666666666666666
P@7 : 0.2857142857142857
P@8 : 0.25
P@9 : 0.2222222222222222
P@10 : 0.3
******* Average Precision : 0.27714285714285714
--------------------------------

0.3517169312169311

## MRR

In [8]:
def mean_reciprocal_rank(relevant_docs, retrieved_docs):
    """
    Calculates Mean Reciprocal Rank (MRR)

    Args:
        relevant_docs: A dictionary mapping query IDs to a list of relevant document IDs.
        retrieved_docs: A dictionary mapping query IDs to a list of retrieved document IDs, ranked by relevance.

    Returns:
        The mean reciprocal rank (MRR) score.
    """
    sum_reciprocal_rank = 0
    num_queries = len(relevant_docs)
    for query_id, relevant in relevant_docs.items():
        rele = [item['doc_id'] for item in relevant]
        if query_id in retrieved_docs:
            retrieved = retrieved_docs[query_id]
            reciprocal_rank = 0
            for rank,retrieved_doc in enumerate(retrieved, 1):
                if retrieved_doc in rele:
                    reciprocal_rank = 1 / rank
                    break
            sum_reciprocal_rank += reciprocal_rank
    return sum_reciprocal_rank / num_queries


mean_reciprocal_rank(qrels,queries)


0.4026819923371647